In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from geopy.geocoders import Nominatim
import pandas as pd
import time
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [2]:
ciudad = "santander"
url = 'https://www.spain.info/es/resultados-busqueda/index.html?q='
url2 = '&reloaded&page=1&rpp=36&subTypeFilter=Museo,Monumento,Jard%C3%ADn%20hist%C3%B3rico'

In [3]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies

opciones.add_argument('--incognito')              # incognito


PATH='./chromedriver'
driver=webdriver.Chrome(PATH, options=opciones)
driver.get(url + ciudad + url2)

In [4]:
texto_visitar = driver.find_elements_by_class_name('title')

In [5]:
frase = []
for i in texto_visitar:
    frase.append(i.text)

frase = frase[11:]

In [6]:
latitud = []
longitud = []
for i in frase:
    try:
        geolocator = Nominatim(user_agent="edu")
        location = geolocator.geocode(i)
        lat = location.latitude
        lon = location.longitude
        latitud.append(lat)
        longitud.append(lon)
    except:
        latitud.append("NaN")
        longitud.append("NaN")

In [7]:
driver.get('https://www.spain.info/es/resultados-busqueda/index.html?q=barcelona&reloaded&page=1&rpp=36&typeFilter=SEG-TUR-AtractivoTuristicoNatural')

In [8]:
texto_naturaleza = driver.find_elements_by_class_name('title')

frase2 = []
for i in texto_naturaleza:
    frase2.append(i.text)

In [9]:
frase2 = frase2[11:20]
frase2

['Parque Natural de la Montaña de Montserrat',
 "Parque Natural de Sant Llorenç del Munt i serra de l'Obac",
 'Parque Natural de las Fuentes Carrionas y Fuente Cobre-Montaña Palentina',
 'Parque Natural de Serres de Cadí-Moixeró',
 'Parque Natural del Montseny']

In [10]:
df = pd.DataFrame(list(zip(frase,latitud,longitud)), columns = ['Visitar',"Latitud","Longitud"])
df = df[df["Latitud"] != "NaN"]
df = df.reset_index(drop=True)
df

,Visitar,Latitud,Longitud
0,Museo Marítimo del Cantábrico,43.4633,-3.78368
1,Museo de Prehistoria y Arqueología de Cantabria,43.4623,-3.804
2,Iglesia Catedral de Nuestra Señora de la Asunción,41.6526,-4.72339


In [11]:
df2 = pd.DataFrame(list(zip(frase2)), columns = ['Naturaleza'])
df2

,Naturaleza
0,Parque Natural de la Montaña de Montserrat
1,Parque Natural de Sant Llorenç del Munt i serr...
2,Parque Natural de las Fuentes Carrionas y Fuen...
3,Parque Natural de Serres de Cadí-Moixeró
4,Parque Natural del Montseny


In [12]:
result = pd.concat([df, df2], axis=1)

In [13]:
result

,Visitar,Latitud,Longitud,Naturaleza
0,Museo Marítimo del Cantábrico,43.4633,-3.78368,Parque Natural de la Montaña de Montserrat
1,Museo de Prehistoria y Arqueología de Cantabria,43.4623,-3.804,Parque Natural de Sant Llorenç del Munt i serr...
2,Iglesia Catedral de Nuestra Señora de la Asunción,41.6526,-4.72339,Parque Natural de las Fuentes Carrionas y Fuen...
3,NaN,NaN,NaN,Parque Natural de Serres de Cadí-Moixeró
4,NaN,NaN,NaN,Parque Natural del Montseny


In [14]:
geolocator = Nominatim(user_agent="edu")
location = geolocator.geocode("santander")
lat = location.latitude
lon = location.longitude

In [15]:
map_2 = Map(location=[f'{lat}',f'{lon}'],zoom_start=15)

In [17]:
for i, row in df.iterrows():
    distrito = {
        "location":[row["Latitud"], row["Longitud"]],
        "tooltip" : row["Visitar"]}
    
    icon = Icon(color = "blue",
             prefix = "fa",
             icon = "street-view",
             icon_color = "black")
    
    Marker(**distrito,icon = icon ).add_to(map_2)
    
map_2